Smoothed surfaces saved at /home/localadmin/hpc_mount/Cortical_Microstructure_Changes_in_Schizophrenia/new_group_analysis/group_{hemi}_ {para}_{size}.mgh

In [ ]:
import numpy as np
import nibabel as nib
from pathlib import Path
import os
import subprocess

def setup_freesurfer():
    """Initialize FreeSurfer environment"""
    freesurfer_home = '/home/localadmin/freesurfer'
    subjects_dir = "/home/localadmin/hpc_mount/Cortical_Microstructure_Changes_in_Schizophrenia"
    
    os.environ['FREESURFER_HOME'] = freesurfer_home
    os.environ['SUBJECTS_DIR'] = subjects_dir
    
    setup_cmd = f"bash -c 'source {freesurfer_home}/SetUpFreeSurfer.sh; env'"
    try:
        process = subprocess.Popen(setup_cmd, stdout=subprocess.PIPE, shell=True)
        output, _ = process.communicate()
        
        for line in output.decode().split('\n'):
            if '=' in line:
                key, value = line.split('=', 1)
                os.environ[key] = value
        
        print(f"FreeSurfer environment initialized successfully")
        print(f"FREESURFER_HOME: {os.environ.get('FREESURFER_HOME')}")
        print(f"SUBJECTS_DIR: {os.environ.get('SUBJECTS_DIR')}")  # Add this line
    except Exception as e:
        print(f"Error setting up FreeSurfer: {e}")
        raise
    
def smooth_mgh(mgh_file, fwhm=6):
    """
    Smooth mgh file using FreeSurfer's mri_surf2surf
    
    Parameters:
    -----------
    mgh_file : str
        Path to input .mgh file
    fwhm : float
        FWHM in mm for smoothing (default=6)
    """
    try:
        print(f"\nProcessing: {mgh_file}")
        
        # Create output filename
        output_dir = os.path.dirname(mgh_file)
        base_name = Path(mgh_file).stem
        output_file = os.path.join(output_dir, f"{base_name}_fwhm{fwhm}.mgh")
        
        # Determine hemisphere from filename
        if 'lh.' in mgh_file or '_lh_' in mgh_file:
            hemi = 'lh'
        elif 'rh.' in mgh_file or '_rh_' in mgh_file:
            hemi = 'rh'
        else:
            raise ValueError("Cannot determine hemisphere from filename")
            
        # Construct the mri_surf2surf command
        cmd = [
            'mri_surf2surf',
            '--s', 'fsaverage',          # Use fsaverage as both source and target
            '--hemi', hemi,              # Specify hemisphere (lh or rh)
            '--sval', mgh_file,          # Source/input values file
            '--fwhm-src', str(fwhm),     # Smooth source with specified FWHM
            '--cortex',                  # Use ?h.cortex.label as smoothing mask
            '--tval', output_file        # Target/output file
            ]
        
        # Run the command
        result = subprocess.run(cmd, 
                              capture_output=True, 
                              text=True, 
                              check=True)
        
        # Verify the output file exists
        if os.path.exists(output_file):
            print(f"Successfully smoothed: {output_file}")
            
            # Load and print basic stats of output for verification
            img = nib.load(output_file)
            data = img.get_fdata()
            print(f"Output shape: {data.shape}")
            print(f"Output mean: {np.mean(data):.6f}")
            print(f"Output std: {np.std(data):.6f}")
            
            return True, output_file
        else:
            raise FileNotFoundError(f"Output file not created: {output_file}")
            
    except subprocess.CalledProcessError as e:
        print(f"FreeSurfer command failed for {mgh_file}:")
        print(f"Error output: {e.stderr}")
        return False, None
    except Exception as e:
        print(f"Error processing {mgh_file}:")
        print(f"Error message: {str(e)}")
        return False, None

def process_all_mgh_files(base_dir, fwhm=6):
    """
    Process all group mgh files in directory with smoothing
    
    Parameters:
    -----------
    base_dir : str
        Directory containing the MGH files
    fwhm : float
        FWHM in mm for smoothing (default=6)
    """
    # Parameters and hemispheres
    params = ['ad', 'md', 'rd', 'ak', 'mk', 'rk', 'fa']
    hemis = ['lh', 'rh']
    
    processed = []
    failed = []
    
    # Process each combination
    for hemi in hemis:
        for param in params:
            mgh_file = os.path.join(base_dir, f"group_{hemi}_{param}.mgh")
            if os.path.exists(mgh_file):
                success, output_file = smooth_mgh(mgh_file, fwhm)
                if success:
                    processed.append(mgh_file)
                else:
                    failed.append(mgh_file)
    
    # Print summary
    print("\nProcessing Summary:")
    print(f"Total files processed successfully: {len(processed)}")
    print(f"Total files failed: {len(failed)}")
    
    if failed:
        print("\nFailed files:")
        for f in failed:
            print(f"  {f}")

# Run the script
if __name__ == "__main__":
    # Make sure SUBJECTS_DIR environment variable is set
    setup_freesurfer()
    if 'SUBJECTS_DIR' not in os.environ:
        raise EnvironmentError("SUBJECTS_DIR environment variable must be set")
        
    base_dir = "/home/localadmin/hpc_mount/Cortical_Microstructure_Changes_in_Schizophrenia/new_group_analysis"
    process_all_mgh_files(base_dir, fwhm=6)
    process_all_mgh_files(base_dir, fwhm=10)